# Support Vector Machine Learning Model (99% variance)

Import required packages

In [1]:
import numpy as np # for multi-dimensional array operations
import pandas as pd # for reading data from .csv files
from sklearn.svm import SVC # for support vector machine model
from sklearn.decomposition import PCA # for principle component analysis (dimensionality reduction)
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, StratifiedKFold # for getting the best hyper parameters

Assign the training set and testing set to variables for easy reference

In [2]:
train_set = pd.read_csv('../train_tfidf_features.csv') # import the training set
test_set = pd.read_csv('../test_tfidf_features.csv') # import the testing set

Principal Component Analysis for train_set (99% variance)

In [3]:
train_set_label = train_set.loc[:, ["label"]]
features_names = [str(i) for i in range(0, 5000)]
train_set_features = train_set.loc[:, features_names] # train_set_features will not contain the label and id columns

# perform PCA
pca = PCA(n_components = 0.99)
train_set_reduced = pca.fit_transform(train_set_features)
train_set_reduced = pd.DataFrame(data = train_set_reduced)
train_set_reduced

,0,1,2,3,4,5,6,7,8,9,...,4429,4430,4431,4432,4433,4434,4435,4436,4437,4438
0,-0.083198,-0.016048,-0.010595,-0.001965,-0.013779,-0.010989,-0.009680,-0.007520,-0.022370,-0.023418,...,0.006473,-0.002981,0.000318,-0.000758,-0.000867,0.000631,0.000602,-0.004018,-0.002612,0.002346
1,-0.068421,-0.043649,-0.018443,-0.008228,-0.000051,-0.043177,0.127322,0.010281,0.014349,-0.005608,...,0.006526,0.004299,0.004485,0.000582,-0.000948,0.009072,-0.001742,0.000369,0.001724,0.002920
2,-0.080171,-0.044642,-0.015342,-0.008697,-0.010481,-0.057562,0.075615,0.115411,0.111499,0.070551,...,-0.013916,0.001134,0.007410,0.008349,-0.010908,-0.007439,0.000891,-0.000969,-0.003461,0.001669
3,0.028600,-0.040406,0.002784,0.007087,0.000462,-0.031219,-0.137443,0.114581,-0.013427,0.077351,...,-0.013115,-0.007898,-0.003619,-0.005280,0.031062,-0.007166,0.003460,0.023657,0.000545,0.000961
4,0.255054,-0.113418,-0.019237,-0.021468,-0.040033,-0.007000,-0.035960,0.008656,-0.026048,-0.003353,...,0.009921,-0.000787,-0.003085,0.003379,0.004497,-0.005915,0.010054,-0.009144,0.004575,-0.003255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17179,0.075930,0.078879,0.167305,-0.099091,0.304845,0.037762,-0.022524,0.018488,-0.015024,-0.006992,...,0.002317,0.002755,0.001834,-0.001933,-0.003613,-0.001562,-0.001220,-0.002336,-0.008255,0.002295
17180,-0.072104,-0.019731,-0.014655,-0.005451,-0.005682,-0.012314,0.005986,-0.009017,-0.003207,-0.014674,...,-0.004103,0.004485,0.002309,-0.007496,-0.003132,-0.002091,0.003855,0.001271,-0.000360,0.002731
17181,0.002079,-0.041786,-0.016886,-0.008919,-0.018063,-0.020858,0.006685,-0.006916,-0.017607,0.010785,...,-0.002536,0.008880,0.021628,0.000106,0.001599,-0.002772,-0.011255,-0.005111,-0.009403,-0.005681
17182,0.091355,-0.055903,-0.008473,-0.017559,-0.002191,-0.013549,-0.012348,-0.003722,-0.017070,-0.010729,...,-0.001412,-0.000991,-0.000126,0.001060,0.004109,0.000742,-0.003055,0.002958,0.000408,-0.000240


In [4]:
X = train_set_reduced
y = train_set_label

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20) # Train a SVC model using different kernel
X_train = X
y_train = y

In [5]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,4429,4430,4431,4432,4433,4434,4435,4436,4437,4438
0,-0.083198,-0.016048,-0.010595,-0.001965,-0.013779,-0.010989,-0.009680,-0.007520,-0.022370,-0.023418,...,0.006473,-0.002981,0.000318,-0.000758,-0.000867,0.000631,0.000602,-0.004018,-0.002612,0.002346
1,-0.068421,-0.043649,-0.018443,-0.008228,-0.000051,-0.043177,0.127322,0.010281,0.014349,-0.005608,...,0.006526,0.004299,0.004485,0.000582,-0.000948,0.009072,-0.001742,0.000369,0.001724,0.002920
2,-0.080171,-0.044642,-0.015342,-0.008697,-0.010481,-0.057562,0.075615,0.115411,0.111499,0.070551,...,-0.013916,0.001134,0.007410,0.008349,-0.010908,-0.007439,0.000891,-0.000969,-0.003461,0.001669
3,0.028600,-0.040406,0.002784,0.007087,0.000462,-0.031219,-0.137443,0.114581,-0.013427,0.077351,...,-0.013115,-0.007898,-0.003619,-0.005280,0.031062,-0.007166,0.003460,0.023657,0.000545,0.000961
4,0.255054,-0.113418,-0.019237,-0.021468,-0.040033,-0.007000,-0.035960,0.008656,-0.026048,-0.003353,...,0.009921,-0.000787,-0.003085,0.003379,0.004497,-0.005915,0.010054,-0.009144,0.004575,-0.003255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17179,0.075930,0.078879,0.167305,-0.099091,0.304845,0.037762,-0.022524,0.018488,-0.015024,-0.006992,...,0.002317,0.002755,0.001834,-0.001933,-0.003613,-0.001562,-0.001220,-0.002336,-0.008255,0.002295
17180,-0.072104,-0.019731,-0.014655,-0.005451,-0.005682,-0.012314,0.005986,-0.009017,-0.003207,-0.014674,...,-0.004103,0.004485,0.002309,-0.007496,-0.003132,-0.002091,0.003855,0.001271,-0.000360,0.002731
17181,0.002079,-0.041786,-0.016886,-0.008919,-0.018063,-0.020858,0.006685,-0.006916,-0.017607,0.010785,...,-0.002536,0.008880,0.021628,0.000106,0.001599,-0.002772,-0.011255,-0.005111,-0.009403,-0.005681
17182,0.091355,-0.055903,-0.008473,-0.017559,-0.002191,-0.013549,-0.012348,-0.003722,-0.017070,-0.010729,...,-0.001412,-0.000991,-0.000126,0.001060,0.004109,0.000742,-0.003055,0.002958,0.000408,-0.000240


In [6]:
y_train

,label
0,1
1,0
2,1
3,0
4,1
...,...
17179,0
17180,0
17181,1
17182,1


Principal Component Analysis for test_set (99% variance)

In [7]:
features_names = [str(i) for i in range(0, 5000)]
test_set_features = test_set.loc[:, features_names] # test_set_features will not contain the label and id columns

# perform PCA
test_set_reduced = pca.transform(test_set_features) # use the pca from the train_set?
test_set_features = pd.DataFrame(data = test_set_reduced)
test_set_features

,0,1,2,3,4,5,6,7,8,9,...,4429,4430,4431,4432,4433,4434,4435,4436,4437,4438
0,-0.082801,-0.009343,-0.013543,-0.013317,-0.007984,-0.002459,-0.026278,0.004096,-0.014072,-0.020606,...,0.004028,0.006925,0.002027,0.001339,-0.000593,0.001723,-0.003954,0.002756,0.002250,0.004846
1,-0.075595,0.005040,-0.017771,-0.014935,-0.000467,-0.001601,-0.055948,-0.017461,-0.031185,-0.017278,...,-0.003954,-0.002161,0.000289,0.005750,-0.000444,-0.004787,0.000496,0.005983,0.003478,0.009520
2,0.163876,0.059162,-0.057706,-0.040884,-0.063164,-0.003223,0.020473,-0.005967,-0.005539,-0.008680,...,0.007354,-0.006961,0.004528,-0.005167,-0.002319,-0.004137,-0.004697,0.002819,0.007392,0.007172
3,-0.006835,0.189357,-0.044389,-0.024452,-0.043835,-0.013077,0.001344,0.000674,-0.005021,-0.028871,...,-0.000363,0.000194,-0.000551,-0.000187,0.004013,-0.002080,0.003053,-0.003873,0.003679,0.000215
4,0.120704,-0.089485,0.015896,0.030057,-0.018015,-0.057111,-0.072387,0.053490,-0.153091,0.026020,...,-0.000542,-0.001976,-0.001583,0.003821,-0.000703,0.001095,-0.001370,0.003472,0.002224,0.002609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4291,-0.076808,-0.024913,-0.014752,-0.008106,-0.009817,-0.013137,0.026288,-0.012828,-0.004845,-0.010727,...,-0.004400,0.008413,0.006682,-0.006936,0.000967,0.001696,-0.005128,-0.001915,-0.000324,-0.003903
4292,0.069331,-0.055078,-0.008249,-0.022183,-0.000882,0.024624,-0.085988,-0.152147,0.159634,0.018128,...,-0.020066,-0.004857,-0.000050,-0.014649,0.019796,0.002825,0.043783,0.006675,-0.003466,-0.018214
4293,0.042192,-0.056123,-0.009015,-0.022608,-0.000348,0.011589,-0.046442,-0.129357,0.126640,0.010080,...,-0.000224,-0.001619,0.000299,-0.001373,0.008846,0.008807,0.001446,-0.011323,-0.002336,-0.017370
4294,0.220417,-0.101562,-0.013420,-0.014971,-0.033857,-0.007979,-0.018445,-0.004240,-0.019587,-0.000660,...,-0.002864,0.034982,0.031452,0.011856,-0.016514,-0.008613,0.022163,-0.009243,0.023390,-0.016125


Tuning the hyper-parameters and training the model based on the best hyper-parameters

In [8]:
# hyper_parameters = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['linear', 'poly', 'sigmoid', 'rbf']} # initialise the hyper-parameters
hyper_parameters = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']} # initialise the hyper-parameters
kfold = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0) # for 3-fold cross validation
# grid = GridSearchCV(SVC(), hyper_parameters, refit = True, verbose = 2) # cretae a GridSearchCV object to git to the taining data
grid = GridSearchCV(SVC(), param_grid = hyper_parameters, scoring = 'accuracy', refit = 'accuracy', n_jobs = 1 , cv = kfold, verbose = 2)
grid.fit(X_train, np.ravel(y_train)) # training the model using the best hyper-parameters
print(grid.best_params_) # gets the best hyper-parameters for SVM

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 9.9min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 9.4min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 9.3min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 9.2min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=10.9min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=10.5min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 9.2min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 9.3min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 9.5min
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time= 8.0min
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time= 8.6min
[CV] END .....................C=0.1, gamma=0.001

Predicting the labels for the test dataset based on the model with the best hyper-parameters

In [9]:
y_predicted = grid.predict(test_set_features)

In [10]:
# y_predicted = svc_model.predict(test_set_features)
y_predicted = pd.DataFrame(y_predicted, columns = ['label']) # convert y_predicted from nparray to pandas dataframe
y_predicted.insert(loc = 0, column = 'id', value = [i for i in range(17185, 17185 + 4296)]) # insert a column of the ids, starting from 17185
y_predicted.to_csv('skynet_submission_99.csv', index = False) # output the predicted labels to ./skynet_submission_99.csv